In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
D_data = pd.read_csv('data/Denmark/Merged.csv')
USA_data = pd.read_csv('data/USA/filtered_USA_votes.csv')

In [3]:
D_data['opdateringsdato_x'] = pd.to_datetime(D_data['opdateringsdato_x'],errors='coerce')

start_date = "2022-12-01"
end_date = "2025-12-31"

D_data = D_data[(D_data['opdateringsdato_x'] >= start_date) & (D_data['opdateringsdato_x'] <= end_date)]

In [ ]:
# Count how many voted 'Yes' (1) and 'No' (2) per (party, rollnumber)
vote_counts = USA_data.groupby(["party_code", "rollnumber", "cast_code"]).size().unstack(fill_value=0)

# Rename columns for clarity
vote_counts.columns = ["no_votes", "yes_votes"]

# Compute total votes per rollnumber per party
vote_counts["total_votes"] = vote_counts["no_votes"] + vote_counts["yes_votes"]

# Compute agreement percentage (max(yes, no) / total)
vote_counts["agreement_rate"] = vote_counts[["no_votes", "yes_votes"]].max(axis=1) / vote_counts["total_votes"]

# Compute variance and standard deviation per party
party_stats = vote_counts.groupby("party_code")["agreement_rate"].agg(["mean", "var", "std"]).reset_index()

# Display results
print(party_stats)

,party_code,rollnumber,cast_code
0,100,1,1
1,100,2,1
2,100,3,1
3,100,4,1
4,100,5,1
...,...,...,...
2465,200,1231,3
2466,200,1232,3
2467,200,1233,3
2468,200,1234,3


In [7]:
grouped_USA = USA_data.groupby("party_code")
vote_stats = grouped_USA["cast_code"].agg(["count", "var", "std"]).reset_index()
vote_stats

,party_code,count,var,std
0,100,263903,2.383902,1.543989
1,200,273240,2.215623,1.488497
